In [65]:
import pandas as pd
import pyodbc
import warnings
from tqdm import tqdm  # Import tqdm for progress bar

# Ignore warnings
warnings.filterwarnings("ignore")

# Define your database connection parameters
server = '192.168.29.215,4133\FELIX_HEALTHCARE'
database = 'Felix_Healthcare'
table_name = 'Pharmacy_IPD_P_Sales_137'  # Replace with your table name
conn_str = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes'

# Load data from the Excel file into a DataFrame
excel_file = r'C:\Users\kumar\Downloads\Patientwise Sales (4).xlsx'
df = pd.read_excel(excel_file)  # Replace 'Data' with your sheet name


In [66]:
# Convert specific columns to datetime and replace the null values

df.iloc[:,0] = pd.to_datetime(df.iloc[:,0])
df.iloc[:,19] = pd.to_datetime(df.iloc[:,19])
df.fillna('-', inplace = True)

In [67]:
# Special characters to remove
special_characters = ["'", "`", "~", "#", "@"]

# Columns you want to clean
columns_to_clean = list(range(1, 6)) + list(range(12, 18)) + [20]

# Loop through selected columns and remove special characters
for col in columns_to_clean:
    for char in special_characters:
        df.iloc[:, col] = df.iloc[:, col].str.replace(f'[{char}\s]', '', regex=True)

In [68]:
# Create a connection to the SQL Server database
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# Insert data into the SQL Server table with a progress bar
with tqdm(total=len(df)) as pbar:
    for index, row in df.iterrows():
        try:
            # Build the SQL INSERT statement dynamically based on column names and values
            columns = "TXN_DATE, SR_NO, PATIENT_NAME, ITEM_NAME, EXPIRY, PACKING, BATCH, UNIT, QTY, AMOUNT, DISCOUNT, PURCHASE_AMT, TYPE, GENDER, MOBILE_PHONE_NO, PATIENT_CATEGORY, MRN, REFERRING_DOCTOR, CASHIER, UPDATED_TIME, REGISTRATION_NO"  # Get all columns from the DataFrame
            values = ", ".join([f"'{value}'" if pd.notna(value) else "NULL" for value in row])
            sql_insert = f"INSERT INTO {table_name} ({columns}) VALUES ({values})"

            # Execute the SQL statement
            cursor.execute(sql_insert)
            conn.commit()

        except Exception as e:
            print(f"Error at row {index + 2}: {e}")  # Adding 2 because the row index is 0-based
            break  # Exit the loop on error

        # Update the progress bar
        pbar.update(1)

# Close the database connection
conn.close()

print("Data Updated")


100%|██████████████████████████████████████████████████████████████████████████| 46963/46963 [00:38<00:00, 1227.74it/s]

Data Updated
